In [8]:
import warnings
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
from pathlib import Path


In [9]:
#process log tracking
logging.basicConfig(level=logging.DEBUG,
                    filename='./logfile.log',
                    filemode='w', # 'w' 表示寫模式, 'a' 表示追加模式, 'w' 表示如果文件已存在，先将其清空。如果你想在不清空现有日志的情况下向文件追加日志，可以使用 'a' 模式。
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger = logging.getLogger(__name__)


In [10]:
#evaluation function
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


## RAW data preparation and split data for training

In [11]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from local
logger.debug("raw data ingestion")
data = pd.read_csv("data/red-wine-quality.csv")
data.to_csv("data/red-wine-quality.csv", index=False)

# Split the data into training and test sets. (0.75, 0.25) split.
logger.debug("data prep for training and testing")
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

## Model training and prediction


### MLflow logging function
1. get tracking uri -> 設定ML實驗儲存位置
2. set tracking uri

In [12]:
# set tracking folder
mlflow.set_tracking_uri(uri="")

# 全路徑寫法 file:xxxx
# mlflow.set_tracking_uri(uri=r"file:C:\Users\xdxd2\Sunny_VS_worksapce\Sunny_python\ML\mytracks")

print("The set tracking uri is ", mlflow.get_tracking_uri())



The set tracking uri is  


1. create_experiment() => Create a fresh new experiment
2. set_experiment() => set already created experiment

In [13]:
# exp = mlflow.set_experiment( experiment_name= "lr_elasticNet_v1")

exp_id = mlflow.create_experiment(
    name="exp_create_exp_artifact",
    tags={"version": "v1", "priority": "p1"},
    artifact_location=Path.cwd().joinpath("myartifacts").as_uri()
)

get_exp = mlflow.get_experiment(exp_id)


print(f"Name: {get_exp.name}")
print(f"Experiment_id: {get_exp.experiment_id}")
print(f"Artifact Location: {get_exp.artifact_location}")
print(f"Tags: {get_exp.tags}")
print(f"Lifecycle_stage: {get_exp.lifecycle_stage}")
print(f"Creation timestamp: {get_exp.creation_time}")




Name: exp_create_exp_artifact
Experiment_id: 704962840716808362
Artifact Location: file:///c:/Users/xdxd2/Sunny_VS_worksapce/Sunny_python/ML/MLOps_fundamentals/MLflow/basic/myartifacts
Tags: {'priority': 'p1', 'version': 'v1'}
Lifecycle_stage: active
Creation timestamp: 1706496347727


In [14]:

with mlflow.start_run(experiment_id=exp_id):

    # hyper parameter tuning
    alpha = 0.3
    l1_ratio = 0.3


    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "mymodel")



Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7442929001520973
  MAE: 0.5763000946156918
  R2: 0.21508707276848893


### MLflow experiment function -2
1. create_experiment() => Create a fresh new experiment
2. set_experiment() => set already created experiment

In [15]:

exp = mlflow.set_experiment( experiment_name= "lr_elasticNet_v1")


print(f"Name: {exp.name}")
print(f"Experiment_id: {exp.experiment_id}")
print(f"Artifact Location: {exp.artifact_location}")
print(f"Tags: {exp.tags}")
print(f"Lifecycle_stage: {exp.lifecycle_stage}")
print(f"Creation timestamp: {exp.creation_time}")



2024/01/29 11:13:39 INFO mlflow.tracking.fluent: Experiment with name 'lr_elasticNet_v1' does not exist. Creating a new experiment.


Name: lr_elasticNet_v1
Experiment_id: 598143468472518456
Artifact Location: file:///C:/Users/xdxd2/Sunny_VS_worksapce/Sunny_python/ML/MLOps_fundamentals/MLflow/basic/mlruns/598143468472518456
Tags: {}
Lifecycle_stage: active
Creation timestamp: 1706498019587


In [17]:
with mlflow.start_run(experiment_id=exp.experiment_id):

    # hyper parameter tuning
    alpha = 0.3
    l1_ratio = 0.3


    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "mymodel")



Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7442929001520973
  MAE: 0.5763000946156918
  R2: 0.21508707276848893
